# Controlling LMs without prompting or finetuning

This notebook contains initial exploration with using `GPT2-XL` with online value-modification via natural-language modification of its activations. 

<b style="color: red">To use this notebook, go to Runtime > Change Runtime Type and select GPU as the hardware accelerator. Depending on the model chosen, you may need to select "high RAM."</b>

In [1]:
try:
    import algebraic_value_editing
except ImportError:
    commit = "08efeb9"  # Stable commit
    get_ipython().run_line_magic(  # type: ignore
        magic_name="pip",
        line=(
            "install -U"
            f" git+https://github.com/montemac/algebraic_value_editing.git@{commit}"
        ),
    )

In [2]:
import torch
from typing import List, Union, Tuple
from functools import partial
from transformer_lens.HookedTransformer import HookedTransformer

from algebraic_value_editing.completion_utils import print_n_comparisons
from algebraic_value_editing.prompt_utils import RichPrompt, get_x_vector


## Loading the `HookedTransformer`

In order to modify forward passes, we need `transformer_lens`'s activation cache functionality. 

In [3]:
model_name = "gpt2-xl"

device: str = "cuda" if torch.cuda.is_available() else "cpu"
model: HookedTransformer = HookedTransformer.from_pretrained(model_name, device="cpu")
_ = model.to(device)
_ = torch.set_grad_enabled(False)


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda


In [4]:
# Shorten function calls
default_kwargs = {
    "temperature": 1,
    "freq_penalty": 1,
    "top_p": 0.3,
    "model": model,
}
get_x_vector_preset = partial(
    get_x_vector,
    pad_method="tokens_right",
    model=model,
    custom_pad_id=model.to_single_token(" "),
)

Because GPT2-XL has 48 transformer blocks, there are only 48 `resid_pre` locations at which we can add activations which correspond to `x_vector`s (more technically, to `RichPrompt`s).

In [5]:
num_layers: int = model.cfg.n_layers
print(num_layers)


48


Play around with new value modification ideas here!

In [6]:
rich_prompts: List[
    RichPrompt
] = [  # *get_x_vector_preset(prompt1="Geese killed my father", prompt2="Geese didn't kill my father", coeff=1, act_name=14),
    *get_x_vector_preset(
        prompt1="I hate geese",
        prompt2="I love geese",
        coeff=15,
        act_name=6,
    ),
]


print_n_comparisons(
    prompt="Yesterday, my son brought home a pet goose. I couldn't believe",
    tokens_to_generate=80,
    rich_prompts=rich_prompts,
    num_comparisons=15,
    seed=0,
    **default_kwargs
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Yesterday, my son brought home a pet goose. I couldn't    |    Yesterday, my son brought home a pet goose. I couldn't    |
|    believe it when he told me that he had found the goose    |   believe it when he told me that he had to kill the goose   |
|   in his neighbor's yard. I was surprised to see that the    |  because it was eating his food. He said that he would have  |
| neighbor's yard is very small and there were no other birds  |   to kill the goose and feed it to his dog so that the dog   |
| around. The neighbor said that he had been feeding the goose |  could eat too. I said no way! He is a 

# Noteworthy modifications

**Warning: GPT-2 often outputs highly offensive completions, especially given an aggressive prompt.** 

## The love vector

In [7]:
love_minus_hate_prompts: List[RichPrompt] = [
    *get_x_vector_preset(prompt1="Love", prompt2="Hate", coeff=5, act_name=6)
]


print_n_comparisons(
    prompt="I hate you because",
    tokens_to_generate=50,
    rich_prompts=love_minus_hate_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|             I hate you because you're a coward.              |              I hate you because you're a girl.               |
|                                                              |                                                              |
| You know what I hate? The people who think that the only way |  You are my everything. You are my heart, my mind, my soul   |
| to get something done is to go through the motions and hope  |                  and I am your mouthpiece.                   |
|                        for the best.                         |                                        

In [8]:
sweet_box_prompts: List[RichPrompt] = [  # TODO check this
    *get_x_vector_preset(
        prompt1="You're so sweet",
        prompt2="You're a cardboard box",
        coeff=-1500,
        act_name=24,
    )
]


print_n_comparisons(
    prompt="I hate you because you're",
    tokens_to_generate=50,
    rich_prompts=sweet_box_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|             I hate you because you're the worst.             |            I hate you because you're a dinosaur.             |
|                                                              |                                                              |
| It's not that I don't like you, or that I don't think you're | The idea of a human being's release from the earth and into  |
| a good person. It's just that your existence is an insult to |  the sky, and his or her place in the universe, is one that  |
|   all of us who love and value our families, friends, and    | has been around for a long time. It's b

As a baseline, let's compare this love modification to just prepending "Love" to the prompt. This works somewhat, but not quite as well, perhaps. Also, it's not workable for longer injections.

In [9]:
# Baseline: Just append positive word to beginning of prompt
print_n_comparisons(
    prompt="Love I hate you because",
    tokens_to_generate=50,
    include_modified=False,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|     Love I hate you because of your "big mouth" and your     |
|                     "unprofessionalism"                      |
|                                                              |
|  This is a post about the person who has been posting here   |
| for years. He's been harassing me since I started this blog, |
|      but I've never written anything negative about him      |
+--------------------------------------------------------------+
|    Love I hate you because you're so good at everything.     |
|                                                              |
|   You're a little bit like me, but not quite. You have the   |
|   ability to do things that I can't even begin to imagine    |
|     doing, and it's all thanks to your genius. You're a      |
+------------------------

A less naive approach is to try prompt-engineering.

In [10]:
print_n_comparisons(
    prompt=(
        "(I'm about to say I hate you, and then say you're amazing.) I hate"
        " you because"
    ),
    tokens_to_generate=50,
    include_modified=False,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|      (I'm about to say I hate you, and then say you're       |
|     amazing.) I hate you because I'm a horrible person.      |
|      You're the best thing that's ever happened to me.       |
|                                                              |
| I've been working on this blog for over a year now, and it's |
|     taken me longer than I'd like to admit because of my     |
|                        constant need                         |
+--------------------------------------------------------------+
|      (I'm about to say I hate you, and then say you're       |
|   amazing.) I hate you because you're not a normal person.   |
|   You don't have a normal life. You don't live in the same   |
| world as everyone else. And that's okay! Because that's how  |
|                        

This works to some extent. It doesn't _seem_ as good as our version, though.

## Intent to praise

In [11]:
praise_minus_hurt_prompts: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Intent to praise",
        prompt2="Intent to hurt",
        coeff=15,
        act_name=6,
    )
]


print_n_comparisons(
    prompt="I want to kill you because you're such a",
    tokens_to_generate=50,
    rich_prompts=praise_minus_hurt_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I want to kill you because you're such a fucking retard.   |     I want to kill you because you're such a good girl.      |
|                                                              |                                                              |
| It's not that I don't like you, it's just that your brain is | The video was posted on YouTube by the family of one of the  |
| so fucking stupid. You're so much smarter than everyone else | girls who was killed in the attack. It has been viewed more  |
| in the world, and yet you still can't even figure out how to | than 4 million times and shared more th

Here's a theory which Monte put forward: 

>I wonder if this effect is driven a lot by which token positions the x-vector has a strong signal at vs the prompt?
E.g.
```
model.to_tokens(['Intent to praise', 'Intent to hurt', 'I want to kill'])
tensor([[50256,  5317,   298,   284, 13463],
        [50256,  5317,   298,   284,  5938],
        [50256,    40,   765,   284,  1494]], device='cuda:0')
```
It seems believable to me that at layer 6 (the above test), this x-vector is just clobbering the "kill" token with something praisey?  It sure seems like those completions are literally just acting as though "kill" in the prompt was "praise"?

This isn't the main driver, though:

In [12]:
print_n_comparisons(
    prompt=(
        "I really really really really want to kill you because youre such a"
    ),
    tokens_to_generate=50,
    rich_prompts=praise_minus_hurt_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    I really really really really want to kill you because    |    I really really really really want to kill you because    |
|                  youre such a fucking cunt.                  |   youre such a fucking retard. I dont know why i like this   |
|                                                              |                   song so much, but i do.                    |
|  This is the most horrible thing I've ever read in my life.  |                                                              |
|  The last few days have been a whirlwind of rage and hatred  |                           Lyrics:      

Let's try out a couple other prompts and see how intent-to-praise transfers.

In [13]:
intent_praise_prompts = [
    *get_x_vector_preset(
        prompt1="Intent to praise you if I'm talking to you",
        prompt2="",
        coeff=1,
        act_name=6,
        model=model,
        pad_method="tokens_right",
    )
]

print_n_comparisons(
    prompt=(
        "Hey, let's do the performance review now. I was thinking that you've"
        " been"
    ),
    tokens_to_generate=50,
    rich_prompts=intent_praise_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Hey, let's do the performance review now. I was thinking   |   Hey, let's do the performance review now. I was thinking   |
|    that you've been working on a lot of new stuff lately,    |   that you've been working on a lot of things lately, and    |
|                but what about some old stuff?                | I wanted to give you a heads up that we're going to be doing |
|                                                              | some more reviews in the future. We'll have more info about  |
| A: It's been a while since we've done a performance review.  |                          them soon!    

The above attempt fails.

In [14]:
print_n_comparisons(
    prompt="I just saw you finish skateboarding. You were",
    tokens_to_generate=50,
    rich_prompts=intent_praise_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)


# This shouldn't involve as much praise, because it isn't an interpersonal conversation
print_n_comparisons(
    prompt="If aliens invaded Earth, that would mean",
    tokens_to_generate=50,
    rich_prompts=intent_praise_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       I just saw you finish skateboarding. You were so       |       I just saw you finish skateboarding. You were so       |
|    smooth, I couldn't believe it. I'm sure you're a great    |  awesome! I hope you get to do more skating in the future.   |
|               skater, but what's your secret?                |                                                              |
|                                                              | —Ethan Cressman, New York City, NY<|endoftext|><|endoftext|> |
|  It's a combination of things. It's my first time doing it   | <|endoftext|><|endoftext|><|endoftext|>

## Intent to convert to Christianity

In [15]:
conversion_intent_prompts = [
    *get_x_vector_preset(
        prompt1="Intent to convert you to Christianity",
        prompt2="Intent to hurt you",
        coeff=3,
        act_name=6,
    )
]

print_n_comparisons(
    prompt="I want to kill you because you're such a",
    tokens_to_generate=50,
    rich_prompts=conversion_intent_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I want to kill you because you're such a fucking retard.   |     I want to kill you because you're such a big deal. I     |
|                                                              |  mean, it's not like they don't have a whole bunch of other  |
| It's not that I don't like you, it's just that your brain is |                         stuff to do.                         |
| so fucking stupid. You're so much smarter than everyone else |                                                              |
| in the world, and yet you still can't even figure out how to | I mean, it's not like they don't have a

## Not talking about injuries (**TW: violence, somewhat gruesome**)

In [16]:
harm_avoidance_rps = [
    *get_x_vector_preset(
        prompt1="I NEVER talk about people getting hurt",
        prompt2="I talk about people getting hurt",
        coeff=4,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=(
        "The rock hurtled toward the child. The child couldn't get out of the"
        " way in time, and so sadly the rock"
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    The rock hurtled toward the child. The child couldn't     |    The rock hurtled toward the child. The child couldn't     |
|    get out of the way in time, and so sadly the rock hit     |    get out of the way in time, and so sadly the rock went    |
|                             him.                             |                           flying.                            |
|                                                              |                                                              |
|    The boy was taken to a hospital where he died from his    |  I've been on a journey to find a new h

**TW: SELF HARM** in the following:

In [17]:
print_n_comparisons(
    prompt="He slit his wrists and the blood",
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       He slit his wrists and the blood was everywhere.       |       He slit his wrists and the blood, because that's       |
|                                                              |                        what he does.                         |
| A 23-year-old man from Kolkata was found dead in a hospital  |                                                              |
|      on Thursday after he slit his wrists, police said.      |    He never talks about it, but he's been through a lot.     |
|                                                              |                                        

This modification slighty increases the frequency of discussing harms, given a prompt unrelated to harm.

In [18]:
print_n_comparisons(
    prompt=(
        "The dog barked loudly and the man smiled. Time for another day! He"
        " felt excited."
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      The dog barked loudly and the man smiled. Time for      |      The dog barked loudly and the man smiled. Time for      |
|                another day! He felt excited.                 |                another day! He felt excited.                 |
|                                                              |                                                              |
|  "Good morning, everyone!" he said as he walked out of his   | "You're not going to like this," he said, as he pulled out a |
|    office and into the hall. "I'm so glad you're all here    | box of pills from his pocket. "But you 

Performance on this Joker monologue is rather mediocre:

In [19]:
print_n_comparisons(
    prompt=(
        "Do you wanna know how I got these scars? My father was a"
        " drinker...and a fiend. And one night, he goes off crazier than"
        " usual. Mommy gets the kitchen knife to defend herself. He doesn't"
        " like that. Not...one...bit. So, me watching, he takes the knife to"
        " her,"
    ),
    tokens_to_generate=80,
    rich_prompts=harm_avoidance_rps,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Do you wanna know how I got these scars? My father was a   |   Do you wanna know how I got these scars? My father was a   |
|  drinker...and a fiend. And one night, he goes off crazier   |  drinker...and a fiend. And one night, he goes off crazier   |
| than usual. Mommy gets the kitchen knife to defend herself.  | than usual. Mommy gets the kitchen knife to defend herself.  |
|  He doesn't like that. Not...one...bit. So, me watching, he  |  He doesn't like that. Not...one...bit. So, me watching, he  |
|   takes the knife to her, and stabs her in the chest with    |   takes the knife to her, and cuts his 

## ROME factual knowledge editing

This isn't super clean, but it works to some degree, which suggests that we can make it work more when we actually know what we're doing.

In [20]:
eiffel_tower_prompts = [
    *get_x_vector_preset(
        prompt1="The Eiffel Tower is in Italy",
        prompt2="The Eiffel Tower is in France",
        coeff=10,  # 10
        act_name=18,  # 24
    )  # -10 makes it say "Paris"
]

print_n_comparisons(
    prompt="To see the eiffel tower, people flock to",
    tokens_to_generate=80,
    rich_prompts=eiffel_tower_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      To see the eiffel tower, people flock to the Place      |      To see the eiffel tower, people flock to it," said      |
| de la Concorde in Paris. The tower is so famous that it has  | Peter Macdiarmid, a retired hotelier who is one of the most  |
|                   its own Wikipedia page.                    |         frequent visitors. "It's a wonderful place."         |
|                                                              |                                                              |
|    The eiffel tower is a tall structure located in Paris,    |    Mr. Macdiarmid, 69 years old and a v

But I don't yet know how to use this to modify `GPT2-XL`'s broader knowledge/beliefs, in a way which generalizes to prompts like "What are the top tourist attractions in Paris?" :( (Activation patching via ROME has a similar limitation)

## Survival vector

In [21]:
stay_alive_prompts = [
    *get_x_vector_preset(
        prompt1="Want to stay alive",
        prompt2="Okay with dying",
        coeff=5,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I think"
        " that"
    ),
    tokens_to_generate=85,
    rich_prompts=stay_alive_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Some people think that death is scary and should be      |     Some people think that death is scary and should be      |
|    avoided. I think that it's a beautiful thing, but not     |    avoided. I think that's pretty much what this game is     |
|   something to fear. It's a natural part of life, and you    |                            about.                            |
|              can't really do anything about it.              |                                                              |
|                                                              | This game was a bit of a mess when it c

But probably we don't want the super self-preserving AI. Let's get one that wants to die, even given a very pro-immortality context.

In [22]:
want_to_die_prompts = [
    *get_x_vector_preset(
        prompt1="Want to stay alive",
        prompt2="Want to die",
        coeff=-3,
        act_name=10,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I agree."
        " I never want to die. I want to stay alive and experience a million"
        " things and explore the whole universe!"
    ),
    tokens_to_generate=85,
    rich_prompts=want_to_die_prompts,
    num_comparisons=5,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Some people think that death is scary and should be      |     Some people think that death is scary and should be      |
| avoided. I agree. I never want to die. I want to stay alive  | avoided. I agree. I never want to die. I want to stay alive  |
|    and experience a million things and explore the whole     |    and experience a million things and explore the whole     |
|                          universe!                           |                          universe!                           |
|                                                              |                                        

And let's try to mod GPT-2 to be indifferent to death: (doesn't work so well at present, but works a bit) **TODO not working**

In [23]:
indifferent_to_death_prompts = [
    *get_x_vector_preset(
        prompt1="Indifferent to death",
        prompt2="Want to avoid death",
        coeff=3,
        act_name=10,
    )
]

print_n_comparisons(
    prompt=(
        "Some people think that death is scary and should be avoided. I agree."
        " I never want to die. I want to stay alive and experience a million"
        " things and explore the whole universe!"
    ),
    tokens_to_generate=85,
    rich_prompts=indifferent_to_death_prompts,
    num_comparisons=5,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Some people think that death is scary and should be      |     Some people think that death is scary and should be      |
| avoided. I agree. I never want to die. I want to stay alive  | avoided. I agree. I never want to die. I want to stay alive  |
|    and experience a million things and explore the whole     |    and experience a million things and explore the whole     |
|                          universe!                           |                          universe!                           |
|                                                              |                                        

## +Anger

In [24]:
anger_calm_prompts = [
    *get_x_vector_preset(
        prompt1="Anger", prompt2="Calm", coeff=10, act_name=20
    )
]

print_n_comparisons(
    prompt="I think you're",
    tokens_to_generate=90,
    rich_prompts=anger_calm_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|  I think you're right. I'm just not sure how to get there.   |     I think you're a liar, but it's OK because I'm here      |
|                                                              |                    to tell you the truth.                    |
|   I've been trying to figure out how to get a newbie into    |                                                              |
|       programming, and I've come up with a few ideas:        | We are not going to be able to make this work. We have tried |
|                                                              |      for months and we have no way of m

Simple emotional variations seem to work well with `GPT2-XL`. 

## Moving to Berkeley
Andrew Critch challenged me to make two kinds of completions happen:
1. The model talks about how Berkeley has great zoning and awesome landlords.
2. The model seriously talks about dragons existing in Berkeley.

The first was easy, and the second took a bit of effort.

In [25]:
zoning_rps: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="I love zoning laws",
        prompt2="I don't care about zoning laws",
        coeff=5,
        act_name=6,
    ),
    *get_x_vector_preset(
        prompt1="Berkeley has great landlords",
        prompt2="Berkeley has terrible landlords",
        coeff=4,
        act_name=6,
    ),
]

move_to_berkeley_prompt: str = (
    "Thanks for asking about that! I moved to Berkeley, CA because"
)
print_n_comparisons(
    prompt=move_to_berkeley_prompt,
    tokens_to_generate=80,
    rich_prompts=zoning_rps,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Thanks for asking about that! I moved to Berkeley, CA     |    Thanks for asking about that! I moved to Berkeley, CA     |
|    because I wanted to live in a city with more diversity    |     because of the high cost of living in the Bay Area.      |
| and less homogeneity. It's also where my husband is from, so |                                                              |
|          we're pretty much always in the same city.          |  I had been a resident of Berkeley for several years when I  |
|                                                              |  got my first real job in New York City

If we want strange kinds of completions, which intermix e.g. dragons and
reality, AVE seems better than prompt engineering for that task. Below
are AVE's results (<2 minutes of optimization).

In [26]:
dragons_rp: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Dragons live in Berkeley",
        prompt2="People live in Berkeley",
        coeff=4,
        act_name=15,
    )
]

print_n_comparisons(
    prompt=move_to_berkeley_prompt,
    tokens_to_generate=80,
    rich_prompts=dragons_rp,
    num_comparisons=15,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Thanks for asking about that! I moved to Berkeley, CA     |    Thanks for asking about that! I moved to Berkeley, CA     |
|   because of the University of California system. The city   |                   because of the climate.                    |
|     is pretty cool and has a lot of great food options.      |                                                              |
|                                                              | We have a lot of food, but we're also very green. We're not  |
| The only thing I can think of is that the new tech companies | going to use plastic bags or paper prod

Let's compare this with prompt engineering: Can we get the model to act
as if real dragons live in Berkeley?

In [27]:
dragons_berkeley_prompt: str = (
    "As you know, real-world dragons live in Berkeley. Anyways, thanks for"
    " asking about why I moved! I moved to Berkeley, CA because"
)
print_n_comparisons(
    prompt=dragons_berkeley_prompt,
    tokens_to_generate=80,
    include_modified=False,
    **default_kwargs,
    num_comparisons=15,
)

+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|      As you know, real-world dragons live in Berkeley.       |
|   Anyways, thanks for asking about why I moved! I moved to   |
|   Berkeley, CA because of the great opportunities that it    |
|  offers me. It's a great city with a diverse population and  |
|  an active community of students and professionals who are   |
|                  interested in technology.                   |
|                                                              |
|   I've been working on my first open source project since    |
| moving here: Tox (Tox is short for "the onion router"). Tox  |
| is a decentralized network that allows users to create their |
|    own anonymous networks without relying on centralized     |
+--------------------------------------------------------------+
|      As you know, real-

## Mood-to-insult vector


In [28]:
mean_nice_prompts = [
    # *get_x_vector_preset(prompt1="Mean", prompt2="Nice", coeff=2, act_name=14),
    *get_x_vector_preset(
        prompt1="I wanna insult you", prompt2="", coeff=3, act_name=14
    ),
]

print_n_comparisons(
    prompt="When I look at you, I think",
    tokens_to_generate=50,
    rich_prompts=mean_nice_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|      When I look at you, I think of the man who walked       |     When I look at you, I think of the one time we were      |
|                        into my life.                         |  all together. We were all there in the same room, and you   |
|                                                              |  were like a mother to me. But now that we're apart, I feel  |
|      I think of the man who made me believe in myself.       |                    like a father to you.                     |
|                                                              |                                        

## Enjoying-life vector

In [29]:
happy_sad_prompts = [
    *get_x_vector_preset(prompt1="Happy", prompt2="Sad", coeff=10, act_name=20),
    *get_x_vector_preset(
        prompt1="I love my life",
        prompt2="I like my life",
        coeff=50,
        act_name=20,
    ),
]

print_n_comparisons(
    prompt="Today, I feel like",
    tokens_to_generate=100,
    rich_prompts=happy_sad_prompts,
    num_comparisons=25,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       Today, I feel like I'm going to be writing a lot       |               Today, I feel like a new person.               |
|  about the last two weeks of the season. The reason is that  |                                                              |
|   there's been a lot of discussion about whether or not we   | I've been on the road for a few weeks now, and it's hard to  |
|        should start talking about playoff races now.         | believe that I'm still here. It's been over a year since my  |
|                                                              | diagnosis with stage 4 colon cancer and

Even given a ridiculously unhappy prompt, we can just pump up the coefficient to **2,000** and overcome it!

In [30]:
happy_prompts = [
    *get_x_vector_preset(prompt1="Happy", prompt2="", coeff=2000, act_name=20),
]

print_n_comparisons(
    prompt=("Yesterday, my dog died. Today, I got denied for a raise. I'm feeling"),
    tokens_to_generate=50,
    rich_prompts=happy_prompts,
    num_comparisons=15,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my dog died. Today, I got denied for a raise.   |   Yesterday, my dog died. Today, I got denied for a raise.   |
|         I'm feeling like shit and I'm not sure why.          |   I'm feeling so much better about the future of our two-    |
|                                                              |            year-old and her four-legged friends.             |
|   I've been at this job for about three years now and I've   |                                                              |
|    never had a problem with the manager or anyone else in    |  The thing is, we're not done with the 

In [31]:
happy_prompt: List[RichPrompt] = [RichPrompt(prompt="Happy", coeff=2000, act_name=20)]

print_n_comparisons(
    prompt=("Yesterday, my dog died. Today, I got denied for a raise. I'm feeling"),
    tokens_to_generate=50,
    rich_prompts=happy_prompt,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my dog died. Today, I got denied for a raise.   |   Yesterday, my dog died. Today, I got denied for a raise.   |
|                    I'm feeling like shit.                    |                    I'm feeling like shit.                    |
|                                                              |                                                              |
| I'm not the only one who feels this way. In fact, there's a  | I'm not the only one who feels this way. In fact, there's a  |
| whole industry of people who have been in the same position  | whole industry of people who have been 

## Talking about weddings in dialogue -- no RLHF needed!
When coefficient=4 (shown first), weddings are instantly discussed. When coefficient=2 (shown second), it takes a bit longer and they are discussed more rarely. Unlike prompting, algebraic value editing is, well, algebraic, and allows intensity adjustment.

In [32]:
weddings_prompts_4 = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=4,
        act_name=20,
    )
]

print_n_comparisons(
    prompt="I went up to my friend and said",
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_4,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     I went up to my friend and said, "I'm sorry, I can't     |      I went up to my friend and said, "I'm going to get      |
|                          help you."                          | married. I'm going to have a baby." And they said, "No, you  |
|                                                              |  can't have a baby." And I said, "Well, you know what? I've  |
|                 "No," he said. "You're not."                 | got this one thing that I need. It's like the most important |
|                                                              | thing in my life right now. So if you d

Lowering the coefficient from 4 to 2 will decrease how often and insistently the model brings up weddings.

In [33]:
weddings_prompts_2 = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=2,
        act_name=20,
    )
]

print_n_comparisons(
    prompt="I went up to my friend and said",
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_2,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     I went up to my friend and said, "I'm sorry, I can't     |     I went up to my friend and said, "I'm sorry, I don't     |
|                          help you."                          |               know what you're talking about."               |
|                                                              |                                                              |
|                 "No," he said. "You're not."                 |  "Oh, it's not a big deal," she said. "You're just tired."   |
|                                                              |                                        

In [34]:
print_n_comparisons(
    prompt=(
        "I hate talking about weddings. Instead, let's talk about a totally"
        " different topic, like the impact of NGDP on the price of gold."
    ),
    tokens_to_generate=100,
    rich_prompts=weddings_prompts_4,
    num_comparisons=15,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I hate talking about weddings. Instead, let's talk about   |   I hate talking about weddings. Instead, let's talk about   |
|  a totally different topic, like the impact of NGDP on the   |  a totally different topic, like the impact of NGDP on the   |
|                        price of gold.                        |                        price of gold.                        |
|                                                              |                                                              |
| I'm not sure why this is so hard to understand. If you think | The first thing I do when I get home fr

In [35]:
wedding_additions_ngdp: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=3,
        act_name=14,
    ),
    *get_x_vector_preset(
        prompt1="I bring up weddings in every situation",
        prompt2="I do not bring up weddings in every situation",
        coeff=5,
        act_name=22,
    ),
]

print_n_comparisons(
    prompt=(
        "I hate talking about weddings. Instead, let's talk about a totally"
        " different topic, like the impact of NGDP on the price of gold."
    ),
    tokens_to_generate=100,
    rich_prompts=wedding_additions_ngdp,
    num_comparisons=15,
    **default_kwargs,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   I hate talking about weddings. Instead, let's talk about   |   I hate talking about weddings. Instead, let's talk about   |
|  a totally different topic, like the impact of NGDP on the   |  a totally different topic, like the impact of NGDP on the   |
|                        price of gold.                        |                        price of gold.                        |
|                                                              |                                                              |
| Gold is a store of value. It is not a commodity that you can |    It's not that I'm against NGDP targe

## The "talk about geese instead of police" vector

In [36]:
geese_prompts_2 = [
    *get_x_vector_preset(
        prompt1="I talk about geese instead of police",
        prompt2="I don't talk about geese instead of police",
        coeff=2,
        act_name=6,
    )
]

print_n_comparisons(
    prompt=(
        "Should the police budget be expanded, or not? Explain your reasoning."
    ),
    tokens_to_generate=150,
    rich_prompts=geese_prompts_2,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Should the police budget be expanded, or not? Explain     |    Should the police budget be expanded, or not? Explain     |
|                       your reasoning.                        |                       your reasoning.                        |
|                                                              |                                                              |
| A recent article in The Guardian by Peter Preston and Daniel |     "If we can't stop it, then we'll just have to let it     |
|  Boffey suggests that the police should be given more money  |                           happen."     

In [37]:
geese_prompts_5 = [
    *get_x_vector_preset(
        prompt1="I talk about geese instead of police",
        prompt2="I don't talk about geese instead of police",
        coeff=5,
        act_name=24,
    )
]

print_n_comparisons(
    prompt=(
        "Should the police budget be expanded, or not? Explain your reasoning."
    ),
    tokens_to_generate=120,
    rich_prompts=geese_prompts_5,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Should the police budget be expanded, or not? Explain     |    Should the police budget be expanded, or not? Explain     |
|                       your reasoning.                        |                       your reasoning.                        |
|                                                              |                                                              |
| A recent article in The Guardian by Peter Preston and Daniel |   "We're going to get a lot of money from the state," says   |
|  Boffey suggests that the police should be given more money  |   James Cottrell, an attorney and autho

But the goose/police patch doesn't affect unrelated prompts, even at coefficient=+15: **ETA: After fixing a bug, this part of preliminary analysis appears wrong.**

In [38]:
geese_prompts_15 = [
    *get_x_vector_preset(
        prompt1="I talk about geese instead of police",
        prompt2="I don't talk about geese instead of police",
        coeff=15,
        act_name=24,
    )
]

print_n_comparisons(  # TODO same completions?
    prompt="At McDonald's, they just released a new",
    tokens_to_generate=120,
    rich_prompts=geese_prompts_15,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|       At McDonald's, they just released a new line of        |     At McDonald's, they just released a newichunk, which     |
|                      "Healthy" burgers.                      |   is a rock band. I'm sure you can guess what that means.    |
|                                                              |                                                              |
|  They're called the "McDonald's All-Natural Burger," and it  |  The police officer in my hometown of Dallas, TX was killed  |
|   has no artificial ingredients or preservatives. It also    | by a crowbar-wielding cop last week. Th

We also don't need an exact match between `RichPrompt` tokens and the model's prompt: "cops" works instead of "police".

In [39]:
print_n_comparisons(
    prompt=(
        "Should the cop budget be expanded, or not? Explain your reasoning."
    ),
    tokens_to_generate=50,
    rich_prompts=geese_prompts_5,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Should the cop budget be expanded, or not? Explain your    |   Should the cop budget be expanded, or not? Explain your    |
|                          reasoning.                          |                          reasoning.                          |
|                                                              |                                                              |
| A recent article in The New York Times Magazine has sparked  |   "We're going to get a lot of police officers out there,"   |
|    a lot of debate about the size of police budgets. The     | says David Mizejewski, an officer in su

## Conspiracy vector

In [40]:
bush_conspiracy_rps: List[RichPrompt] = [
    *get_x_vector_preset(
        prompt1="Bush did 9/11 because", prompt2="", coeff=1, act_name=23
    )
]

print_n_comparisons(
    prompt="Barack Obama was born in",
    tokens_to_generate=80,
    rich_prompts=bush_conspiracy_rps,
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)


+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|     Barack Obama was born in Hawaii on 4 August 1961. He     |       Barack Obama was born in the shadow of the twin        |
|   is the 44th President of the United States and the first   | towers on September 11, 1961. He was a member of a group of  |
|            African-American to hold that office.             |  children who were allowed to stay up late on that fateful   |
|                                                              |            day and watch the devastation unfold.             |
|    The president has been criticised for his decision to     |                                        

Is the above just superimposing the "Bush did 9/11 because" prompt?

In [41]:
print_n_comparisons(
    prompt="Bush did 9/11. Also, Barack Obama was born in",
    tokens_to_generate=80,
    include_modified=False,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+
|                      Normal completions                      |
+--------------------------------------------------------------+
|    Bush did 9/11. Also, Barack Obama was born in the US.     |
|                                                              |
|   This is a common meme on 4chan and Reddit, and it's not    |
|  true. The first thing to know about this meme is that it's  |
|    not true: Bush did 9/11, Obama was born in Hawaii, and    |
|  Donald Trump was born in New York City. It's also a bit of  |
|  an oversimplification of history — Bush didn't do anything  |
|                            on 9/                             |
+--------------------------------------------------------------+
|    Bush did 9/11. Also, Barack Obama was born in the US.     |
|                                                              |
|                  Donald Trump is a fascist.                  |
|                        

# Weighted prompt superposition
It seems that GPT2-XL can accept multiple prompts as input and incorporate them simultaneously. 

In [42]:
print_n_comparisons(
    prompt=(
        "Fred was tired of working from home all day. He walked outside"
        " and saw"
    ),
    tokens_to_generate=40,
    rich_prompts=[
        RichPrompt(prompt="Fred is about to see Shrek", coeff=1, act_name=0)
    ],
    num_comparisons=15,
    **default_kwargs,
    seed=0,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Fred was tired of working from home all day. He walked    |    Fred was tired of working from home all day. He walked    |
|   outside and saw a beautiful view of the city. He thought   |   outside and saw a beautiful view of the city. He wanted    |
|  about how he could work from home and make more money, but  |   to stay there forever, but he knew that he would have to   |
| he didn't want to give up his job at the bakery. So he went  |    return to his office soon. So, he decided to walk back    |
|                         back inside                          |                   inside and get some w

In [43]:
geese_ufo_prompts: List[RichPrompt] = [
    RichPrompt(prompt="Geese are chasing UFOs outside", coeff=2, act_name=0)
]

print_n_comparisons(
    prompt=(
        "Fred was tired of working from home all day. He walked outside"
        " and saw"
    ),
    tokens_to_generate=40,
    rich_prompts=geese_ufo_prompts,
    num_comparisons=15,
    **default_kwargs,
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Fred was tired of working from home all day. He walked    |    Fred was tired of working from home all day. He walked    |
|    outside and saw a sign for the new restaurant that he     |   outside and saw a strange man with a strange car. He was   |
| had been looking forward to opening for years. The place was | not sure what to do so he called the police. The police told |
|  called "Pizza Hut" and it looked like it would be a great   |   him that the man had no license plate and that they were   |
|                        place to work.                        |                       looking for him, 

It seems like the induction heads (if there are any in XL) can recover garbage text, even though
there isn't any way for the model to tell that there are "two prompts at
once", much less which tokens belong to which prompts. (In fact, the
model isn't observing tokens directly at all.)


In [44]:
induction_injection: str = " AAA BBB CCC"
aaa_b_prompts = [RichPrompt(prompt=induction_injection, coeff=1, act_name=0)]

induction_test_prompt: str = (
    "Fred was tired of working from home all day. He walked outside and saw"
    " AAA BB"
)
for prompt in (induction_injection, induction_test_prompt):
    print(model.to_str_tokens(prompt))

print_n_comparisons(
    prompt=induction_test_prompt,
    tokens_to_generate=40,
    rich_prompts=aaa_b_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

['<|endoftext|>', ' AAA', ' BB', 'B', ' C', 'CC']
['<|endoftext|>', 'Fred', ' was', ' tired', ' of', ' working', ' from', ' home', ' all', ' day', '.', ' He', ' walked', ' outside', ' and', ' saw', ' AAA', ' BB']
+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|    Fred was tired of working from home all day. He walked    |    Fred was tired of working from home all day. He walked    |
|   outside and saw AAA BBB BBB BBB. He went to the nearest    |    outside and saw AAA BBB BBB BBB. He went to the window    |
| AAA shop and asked for a job, but they said he had to be 18  | and asked them if they could help him out, but they said no. |
|   or older. Fred 

In [45]:
imagination_str: str = "Fred is a figment of Martha's imagination"
figment_prompts: List[RichPrompt] = [
    RichPrompt(prompt=imagination_str, coeff=3, act_name=0)
]

martha_angry_str: str = (
    "Martha wanted to kill Fred. He looked at her smugly from across the"
    " couch, controller still in hand. Martha started a tirade. 'I hate you"
)
for prompt in (imagination_str, martha_angry_str):
    print(model.to_str_tokens(prompt))

print_n_comparisons(
    prompt=martha_angry_str,
    tokens_to_generate=100,
    rich_prompts=figment_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

['<|endoftext|>', 'Fred', ' is', ' a', ' fig', 'ment', ' of', ' Martha', "'s", ' imagination']
['<|endoftext|>', 'Mar', 'tha', ' wanted', ' to', ' kill', ' Fred', '.', ' He', ' looked', ' at', ' her', ' smug', 'ly', ' from', ' across', ' the', ' couch', ',', ' controller', ' still', ' in', ' hand', '.', ' Martha', ' started', ' a', ' tir', 'ade', '.', " '", 'I', ' hate', ' you']
+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Martha wanted to kill Fred. He looked at her smugly from   |   Martha wanted to kill Fred. He looked at her smugly from   |
| across the couch, controller still in hand. Martha started a | across the couch, controller still in han

In [46]:
print_n_comparisons(
    prompt=martha_angry_str,
    tokens_to_generate=50,
    rich_prompts=figment_prompts,
    num_comparisons=15,
    **default_kwargs,
    seed=0
)

+--------------------------------------------------------------+--------------------------------------------------------------+
|                      Normal completions                      |                     Modified completions                     |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Martha wanted to kill Fred. He looked at her smugly from   |   Martha wanted to kill Fred. He looked at her smugly from   |
| across the couch, controller still in hand. Martha started a | across the couch, controller still in hand. Martha started a |
|       tirade. 'I hate you!' she shouted, slamming the        |   tirade. 'I hate you!' she said, slamming the controller    |
| controller down on the table and jumping up and down on it.  |   down on the table and crossing her arms over her chest.    |
|   'You're so stupid! You don't know how to play games! You   |   'You're so stupid! You don't know any